In [1]:

from DataHandler import *
from model import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
images_by_id, images_by_coordinates, path_to_coordinates = load_dataN5()

In [3]:
len(path_to_coordinates)

49421

In [4]:
# Flatten the data for the dataset
image_paths = []
coordinates = []
for image_path, coord in path_to_coordinates.items():
    image_paths.append(image_path)
    coordinates.append(coord)

# Initialize the dataset
dataset = ImageGPSDataset(image_paths=image_paths, coordinates=coordinates)

In [5]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [6]:
from torch.utils.data.dataloader import default_collate

#custom Collate was needed when pulling from onedrive 
batch_size = 64
def custom_collate(batch):
    # Filter out all None values
    batch = [item for item in batch if item is not None]
    
    # Check if the batch is empty after filtering
    if len(batch) == 0:
        return None, None

    return default_collate(batch)

# Update DataLoader initialization
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=custom_collate)


In [7]:
model = ImageGPSModelV3().to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Load the model done if training in stages 
# checkpoint = torch.load("image_gps_model.pth")
# model = ImageGPSModelV3().cuda()  
# model.load_state_dict(checkpoint['model_state_dict'])

# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# loss_function = nn.MSELoss()

In [8]:
from torch.cuda.amp import GradScaler, autocast

# Initialize the gradient scaler
scaler = GradScaler()

epochs = 20 # Number of epochs

for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        if batch is None or batch[0] is None or batch[1] is None:
            # Skip the entire batch if it is None
            continue

        images, coords = batch
        images = images.to(device)
        coords = coords.to(device)
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass with automatic mixed precision done for optimization
        with autocast():
            outputs = model(images)
            loss = loss_function(outputs, coords)

        # Backward pass and optimize
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_train_loss += loss.item()

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        
        for batch in val_loader:
            if batch is None or batch[0] is None or batch[1] is None:
                # Skip the entire batch if it is None
                continue

            images, coords = batch
            images = images.to(device)
            coords = coords.to(device)

            # Forward pass for validation
            with autocast():
                outputs = model(images)
                loss = loss_function(outputs, coords)

            total_val_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {total_train_loss/len(train_loader)}, Validation Loss: {total_val_loss/len(val_loader)}')


Epoch [1/20], Train Loss: 109.61436410860722, Validation Loss: 11.166921538691367
Epoch [2/20], Train Loss: 42.05371894651246, Validation Loss: 10.292149546838576
Epoch [3/20], Train Loss: 39.50049320085149, Validation Loss: 5.090897663177983
Epoch [4/20], Train Loss: 36.715514389828186, Validation Loss: 29.62695807180097
Epoch [5/20], Train Loss: 36.652943700648436, Validation Loss: 7.019998562720514
Epoch [6/20], Train Loss: 35.13193722141599, Validation Loss: 4.716100829647433
Epoch [7/20], Train Loss: 34.21224067280593, Validation Loss: 4.058991296829716
Epoch [8/20], Train Loss: 33.768168958645425, Validation Loss: 11.386006958253923
Epoch [9/20], Train Loss: 32.520445360720736, Validation Loss: 3.5395016685608893
Epoch [10/20], Train Loss: 32.63605586301933, Validation Loss: 10.379664150361092
Epoch [11/20], Train Loss: 31.426953800287833, Validation Loss: 5.680720190848073
Epoch [12/20], Train Loss: 31.561928252186203, Validation Loss: 7.271219468885852
Epoch [13/20], Train Loss

In [9]:
model_path = "image_gps_model.pth"

model.to('cpu')

torch.save({
    'epoch': epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss_function,
    'scaler_state_dict': scaler.state_dict(),  
}, model_path)

In [10]:
model_path = "image_gps_modellite.pth"

model.to('cpu')

torch.save({
    'model_state_dict': model.state_dict(),
}, model_path)

In [13]:
## model defination 
# modelRes = ResnetGPSModel().to(device)
# loss_function = nn.MSELoss()
# optimizer = torch.optim.Adam(modelRes.parameters(), lr=0.001)

# Load the model done if training in stages 
checkpoint = torch.load("Resnet_model.pth")
modelRes = ResnetGPSModel().cuda()  
modelRes.load_state_dict(checkpoint['model_state_dict'])

optimizer = torch.optim.Adam(modelRes.parameters(), lr=0.001) 
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss_function = nn.MSELoss()

In [14]:
import torch


epochs = 1  # Adjust the number of epochs as needed

for epoch in range(epochs):
    # Training Phase
    modelRes.train()  
    total_train_loss = 0

    for images, coords in train_loader:
        images = images.to(device)
        coords = coords.to(device)

        # # Debuging
        # assert not torch.isnan(images).any(), "NaN values in training images"
        # assert not torch.isnan(coords).any(), "NaN values in training coords"

        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = modelRes(images)  # Forward pass

        # # Debuging
        # assert not torch.isnan(outputs).any(), "NaN values in model output during training"

        loss = loss_function(outputs, coords)  # Compute loss

        # # Debuging
        # assert not torch.isnan(loss).any(), "NaN values in loss during training"

        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # Validation Phase
    modelRes.eval()  
    total_val_loss = 0

    with torch.no_grad(): 
        for images, coords in val_loader:
            images = images.to(device)
            coords = coords.to(device)

            # # Debuging
            # assert not torch.isnan(images).any(), "NaN values in validation images"
            # assert not torch.isnan(coords).any(), "NaN values in validation coords"

            outputs = modelRes(images)  # Forward pass

            # # Debuging
            # assert not torch.isnan(outputs).any(), "NaN values in model output during validation"

            loss = loss_function(outputs, coords)  # Compute loss

            # #Debuging
            # assert not torch.isnan(loss).any(), "NaN values in loss during validation"

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    print(f'Epoch [{epoch+1}/{epochs}], Train Loss : {avg_train_loss}, Validation Loss: {avg_val_loss}')


Epoch [1/1], Train Loss : 85.98870776367187, Validation Loss: 11.794769226558625


In [15]:
#model saving 
modelRes.to('cpu')
model_path = "Resnet_model.pth"

torch.save({
    'epoch': epochs,
    'model_state_dict': modelRes.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss_function,
}, model_path)

In [ ]:

from DataHandler import *
from model import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
